In [19]:
# you can write to stdout for debugging purposes, e.g.
# print("this is a debug message")

def solution(A):
    # Implement your solution here
    N = len(A)
    A.sort()
    i = 0
    while (i <= N-1 and A[i] <= 0):
        i+=1
    #print(A)
    #print("i=",i)
    if (i == N):
        return 1
    else:
        if A[i] > 1:
                return 1
        else:
            if (i == N-1):
                return A[i] + 1
            else:
                while (i+1 <= N-1):
                    #print("i=",i)
                    if (A[i+1] > A[i] + 1):
                        return A[i] + 1
                    i = i + 1
                return A[-1] + 1

solution([-1, -3])

1

In [62]:
import numpy as np
import pandas as pd


# you can use this table as an example
distr_table = pd.DataFrame({
    'X': [0, 0, 1, 1],
    'Y': [1, 2, 1, 2],
    'pr': [0.3, 0.25, 0.15, 0.3]
})

class CheckIndependence:

    def __init__(self):
        self.version = 1
        
    def are_indep(self, joint_distrib, X_distrib, Y_distrib):
        otv=True
        for a in X_distrib:
            for b in Y_distrib:
                if joint_distrib[a+b]!=X_distrib[a]*Y_distrib[b]:
                    otv=False
                    break
        return otv
    
    def EXY(self, joint_distrib, X_distrib, Y_distrib):
        s=0
        for a in X_distrib:
            for b in Y_distrib:
                s+=joint_distrib[a+b]*float(a)*float(b)
                    
        return s
    
    def Eksi(self, Ksi_distrib):
        s=0
        for ksi in Ksi_distrib:
            s+=Ksi_distrib[ksi]*float(ksi)
                    
        return s
    
    def Eksi_kv(self, Ksi_distrib):
        s=0
        for ksi in Ksi_distrib:
            s+=Ksi_distrib[ksi]*float(ksi)*float(ksi)
                    
        return s
    
    def covXY(self, joint_distrib, X_distrib, Y_distrib):
        covXY=self.EXY(joint_distrib, X_distrib, Y_distrib)-self.Eksi(X_distrib)*self.Eksi(Y_distrib)
        return covXY
    
    def corrXY(self, joint_distrib, X_distrib, Y_distrib):
        covXY=self.EXY(joint_distrib, X_distrib, Y_distrib)-self.Eksi(X_distrib)*self.Eksi(Y_distrib)
        DX=self.Eksi_kv(X_distrib) - self.Eksi(X_distrib)*self.Eksi(X_distrib)
        DY=self.Eksi_kv(Y_distrib) - self.Eksi(Y_distrib)*self.Eksi(Y_distrib)
        corrXY=covXY/(np.sqrt(DX)*np.sqrt(DY))
        return corrXY

    def check_independence(self, distr_table: pd.DataFrame):
        # write your solution here
        joint_distrib=dict()
        for i in range(len(distr_table)):
            joint_distrib[str(distr_table['X'][i])+str(distr_table['Y'][i])]=distr_table['pr'][i]
        X_distrib=dict()
        Y_distrib=dict()
        for x in np.unique(distr_table["X"]):
            X_distrib[str(x)]=sum(distr_table[distr_table['X']==x]['pr'])
        for y in np.unique(distr_table["Y"]):
            Y_distrib[str(y)]=sum(distr_table[distr_table['Y']==y]['pr'])
        indep=self.are_indep(joint_distrib, X_distrib, Y_distrib)
        cov=self.covXY(joint_distrib, X_distrib, Y_distrib)
        corr=self.corrXY(joint_distrib, X_distrib, Y_distrib)
        return dict({'are_independent':indep,'cov':cov,'corr':corr})

MyClass=CheckIndependence()
MyClass.check_independence(distr_table)

{'are_independent': False,
 'cov': 0.0525000000000001,
 'corr': 0.21212121212121252}

In [217]:
import warnings

warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV


def regression(data_train, data_test):
    X_train=data_train.loc[:,list(data_train.columns[data_train.columns!='price'])]
    y_train=data_train['price'].values
    X_test=data_test.loc[:,list(data_test.columns[data_test.columns!='price'])]
    y_test=data_test['price'].values
    def  lasso():
        MyLassoCV=LassoCV(alphas=np.logspace(-4, -1, 4),normalize=True)
        MyLassoCV.fit(X_train,y_train)
        alpha=MyLassoCV.alpha_
        y_pred=MyLassoCV.predict(X_test)
        y_pred=np.round(y_pred, decimals=2)
        coef=pd.DataFrame({'variable':X_train.columns, 'coef':MyLassoCV.coef_})
        coef=coef[coef['coef']!=0]
        coef.index=np.arange(len(coef))
        return dict({"alpha":alpha,"pred":y_pred,"coefficients":coef})
    def  ridge():
        MyRidgeCV=RidgeCV(alphas=np.logspace(-4, -1, 4),normalize=True)
        MyRidgeCV.fit(X_train,y_train)
        alpha=MyRidgeCV.alpha_
        y_pred=MyRidgeCV.predict(X_test)
        y_pred=np.round(y_pred, decimals=2)
        coef=pd.DataFrame({'variable':X_train.columns, 'coef':MyRidgeCV.coef_})
        coef=coef[abs(coef['coef'])>0.001]
        coef.index=np.arange(len(coef))
        return dict({"alpha":alpha,"pred":y_pred,"coefficients":coef})
    def  elasticnet():
        MyElasticNetCV=ElasticNetCV(alphas=np.logspace(-4, -1, 4),normalize=True,l1_ratio=np.arange(0.6, 1, 0.1))
        MyElasticNetCV.fit(X_train,y_train)
        alpha=MyElasticNetCV.alpha_
        l1_ratio=MyElasticNetCV.l1_ratio_
        y_pred=MyElasticNetCV.predict(X_test)
        y_pred=np.round(y_pred, decimals=2)
        coef=pd.DataFrame({'variable':X_train.columns, 'coef':MyElasticNetCV.coef_})
        coef=coef[abs(coef['coef'])>0.001]
        coef.index=np.arange(len(coef))
        return dict({"alpha":alpha, "l1_ratio":l1_ratio, "pred":y_pred,"coefficients":coef})
    lasso=lasso()
    ridge=ridge()
    elasticnet=elasticnet()
    print(ridge['alpha'])
    return {'ridge': lasso,
            'lasso': ridge,
            'elastic_net': elasticnet
            }

data_train=pd.read_csv('data_train.csv')
data_test=pd.read_csv('data_test.csv')
# X_train=data_train.loc[:,list(data_train.columns[data_train.columns!='price'])]
# y_train=data_train['price'].values
# X_test=data_test.loc[:,list(data_test.columns[data_test.columns!='price'])]
# y_test=data_test['price'].values

#print(data_train.shape, X_train.shape,y_train.shape )
#print(data_test.shape, X_test.shape,y_test.shape )

regression(data_train, data_test)['ridge']


0.001


{'alpha': 0.01,
 'pred': array([  6.22, 531.25, 377.35, 134.61, 154.21, 102.41, 362.97, 263.1 ,
        334.38, 167.29]),
 'coefficients':    variable       coef
 0       S_1  -0.003165
 1       S_2   0.011310
 2       S_3   1.363747
 3       S_4  -0.011285
 4       S_5  13.108086
 5       S_6   0.000097
 6      S_10   4.660084
 7      S_14   0.007331
 8      S_16   0.019221
 9      S_17  -0.000392
 10     S_18   0.003064
 11     S_19  -0.002484
 12     S_28  -5.570983
 13     S_30  -0.001902
 14     S_33   0.000391
 15     S_34   2.228081
 16     S_41  -0.003377
 17     S_42   0.000185
 18     S_47   6.825045
 19     S_49  -0.003673
 20     S_52   0.000138
 21     S_62   0.003197
 22     S_64  -0.275039
 23     S_65  -3.863479
 24     S_70   0.000805
 25     S_71   0.003954
 26     S_73   0.002747
 27     S_80   0.001015
 28     S_83   0.886578
 29     S_86  -0.183654
 30     S_90  -0.027394
 31     S_91   0.004210
 32     S_92  -0.000994
 33     S_95   0.000515}

In [222]:
import datetime
from pmdarima import arima
df_initial=pd.read_csv("./data/temperatures.csv",parse_dates=[0])
dates=pd.date_range(start='1980-01-01', end='2012-12-2', closed='left',freq=pd.offsets.MonthBegin(1))
df_dates=pd.DataFrame({'ds':dates}) #это объекты типа Timestamp
df=df_initial.merge(df_dates, on='ds', how='outer').sort_values(by='ds').reset_index(drop=True)
y_train=df['y'][:-24]
y_test=df['y'][-24:]



ModuleNotFoundError: No module named 'pmdarima'

In [225]:
!pip3 install pmdarima

xcrun: error: invalid active developer path (/Library/Developer/CommandLineTools), missing xcrun at: /Library/Developer/CommandLineTools/usr/bin/xcrun


In [226]:
from pmdarima.arima import auto_arima

ModuleNotFoundError: No module named 'pmdarima'